In [1]:
import pandas as pd
import requests
import time
from pathlib import Path

In [2]:
# === Config ===
MODEL = "model-1"
EVAL_FILE = "../../data/processed/bdm-corpus-2/stage-4/test.csv"
TRAIN_FILE = "../../data/processed/bdm-corpus-2/stage-4/train.csv" # grab 6 examples for few shot
RESULT_FILE = "../../results/exp003/results.csv"
TEMPERATURE = 0.0

In [3]:
test_df = pd.read_csv(EVAL_FILE)
train_df = pd.read_csv(TRAIN_FILE)

In [4]:
# sample and sort few-shot examples, all occur chronologically before test set
fewshot_up = train_df[train_df["Direção"] == "Aumento"].sample(n=3, random_state=1)
fewshot_down = train_df[train_df["Direção"] == "Diminuição"].sample(n=3, random_state=1)
fewshot_df = pd.concat([fewshot_up, fewshot_down]).copy()
fewshot_df["DataHora"] = pd.to_datetime(fewshot_df["DataHora"])
fewshot_df = fewshot_df.sort_values("DataHora").reset_index(drop=True)

In [5]:
# Format few-shot block
def format_example(row):
    return f"""Manchete: {row['Manchete']}
Resposta: {row['Direção']}"""

FEWSHOT_BLOCK = "\n\n".join(format_example(row) for _, row in fewshot_df.iterrows())

In [6]:
# Build prompt for each headline
def build_fewshot_prompt(headline):
    return f"""{FEWSHOT_BLOCK}

    Preveja o impacto de curto prazo no USD/BRL:

    Manchete: {headline}

    Responda apenas com uma palavra: Aumento ou Diminuição."""

In [7]:
# Query Ollama
def query_ollama(model, prompt):
    res = requests.post("http://localhost:11434/api/chat", json={
        "model": model,
        "temperature": TEMPERATURE,
        "stream": False,
        "messages": [{"role": "user", "content": prompt}]
    })
    res.raise_for_status()
    return res.json()["message"]["content"].strip()

In [8]:
# Run inference
predictions = []
for i, row in test_df.iterrows():
    prompt = build_fewshot_prompt(row["Manchete"])
    try:
        out = query_ollama(MODEL, prompt)
    except Exception:
        out = "Erro"
    predictions.append(out)
    time.sleep(0.25)

In [9]:
test_df["Previsão"] = predictions
test_df.to_csv(RESULT_FILE, index=False)
print(f"✅ Saved predictions to {RESULT_FILE}")

✅ Saved predictions to ../../results/exp003/results.csv


In [10]:
from sklearn.metrics import classification_report

# Reload the results file
preds = pd.read_csv("../../results/exp003/results.csv")

# Normalize predictions (just in case)
preds["Previsão"] = preds["Previsão"].str.strip().str.capitalize()
preds["Direção"] = preds["Direção"].str.strip().str.capitalize()

# Report
report = classification_report(
    preds["Direção"], preds["Previsão"],
    labels=["Aumento", "Diminuição"],
    target_names=["Aumento", "Diminuição"],
    digits=3
)

print("\nClassification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

     Aumento      0.547     0.612     0.578       227
  Diminuição      0.397     0.335     0.364       173

    accuracy                          0.492       400
   macro avg      0.472     0.474     0.471       400
weighted avg      0.482     0.492     0.485       400



In [14]:
from sklearn.metrics import confusion_matrix
'''
[[TP FN]
 [FP TN]]
'''
cm = confusion_matrix(test_df["Direção"], test_df["Previsão"], labels=["Aumento", "Diminuição"])
print(cm)

[[139  88]
 [115  58]]
